In [1]:
from pytorch_lightning import Trainer
import active_catinous.FastGramDynamicMemoryBrainAge as braincatsmodel
from active_catinous.FastGramDynamicMemoryBrainAge import FastGramDynamicMemoryBrainAge
from datasets.BrainAgeDataset import BrainAgeDataset
from datasets.BrainAgeContinuous import BrainAgeContinuous

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import pickle
from py_jotools import mut, slurm, cache
import numpy as np
import gc

import hashlib
import dill
import active_catinous.utils as cutils

from sklearn.metrics import mean_absolute_error

from models.AgePredictor import EncoderRegressor
from models.unet3d import EncoderModelGenesis

%load_ext autoreload
%autoreload 

# BALANCED DATASET TRAINING

In [6]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113 
# 1/10 = 170
# 1/8 = 212 with 0.00001 and 0.0001
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory':True,
        'datasetfile': '/project/catinous/brainds_split_val_test_balance.csv',
        'base_model': 'batch_brainds_split_val_test_balance_base_train_1_88ef2d0cc2.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 64,
        'force_misclassified':True,
        'completion_limit': 7.0, #performance for first domain after base training
        'batch_size': 2,
        'training_batch_size': 8,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'len_perf_queue': 8,
         'transition_phase_after':0.7
        }

startbugdets = [85, 170, 212, 340]
memorymaximas = [64, 128]

for i in range(3):
    for sb in startbugdets:
        for mm in memorymaximas:
            hparams['startbudget'] = sb
            hparams['memorymaximum'] = mm
            hparams['run_postfix'] = 1+i

            if schedule:
                slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
            else:
                model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

activecatinous58556
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous58556.job
Submitted batch job 3058466

activecatinous97184
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous97184.job
Submitted batch job 3058467

activecatinous68393
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous68393.job
Submitted batch job 3058468

activecatinous43918
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous43918.job
Submitted batch job 3058469

activecatinous32534
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous32534.job
Submitted batch job 3058470

activecatinous79908
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/act

In [2]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113
# 1/10 = 170
# 1/8 = 212
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'naive_activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
         'naive_continuous_rate':15,
        'datasetfile': '/project/catinous/brainds_split_val_test_balance.csv',
        'base_model': 'batch_brainds_split_val_test_balance_base_train_1_88ef2d0cc2.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 32,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'transition_phase_after':0.7
        }


continuous_rate = [20, 10, 8, 5]
memorymaximas = [64, 128]

for i in range(3):
    for cr in continuous_rate:
        for mm in memorymaximas:
            hparams['naive_continuous_rate'] = cr
            hparams['memorymaximum'] = mm
            hparams['run_postfix'] = i+1
            if schedule:
                slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
            else:
                model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

naive_activecatinous02159
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous02159.job
Submitted batch job 3058491

naive_activecatinous02699
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous02699.job
Submitted batch job 3058492

naive_activecatinous01759
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous01759.job
Submitted batch job 3058493

naive_activecatinous21932
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous21932.job
Submitted batch job 3058494

naive_activecatinous63852
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous63852.job
Submitted batch job 3058495

naive_activecatinous47396
sshpass -f ~/.ssh/pass ssh cn1.cir.medu

# OLD TRAINING RUNS

In [2]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113
# 1/10 = 170
# 1/8 = 212
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'naive_activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
         'naive_continuous_rate':15,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_val_test_balance_base_train_1_88ef2d0cc2.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 32,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'transition_phase_after':0.7
        }


continuous_rate = [20, 10, 8, 5]
memorymaximas = [64, 128]

for i in range(3):
    for cr in continuous_rate:
        for mm in memorymaximas:
            hparams['naive_continuous_rate'] = cr
            hparams['memorymaximum'] = mm
            hparams['run_postfix'] = i+1
            if schedule:
                slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
            else:
                model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

naive_activecatinous18009
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous18009.job
Submitted batch job 3045284

naive_activecatinous80834
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous80834.job
Submitted batch job 3045285

naive_activecatinous29483
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous29483.job
Submitted batch job 3045286

naive_activecatinous76348
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous76348.job
Submitted batch job 3045288

naive_activecatinous58108
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous58108.job
Submitted batch job 3045289

naive_activecatinous42727
sshpass -f ~/.ssh/pass ssh cn1.cir.medu

In [2]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113
# 1/10 = 170
# 1/8 = 212
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'naive_activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
         'naive_continuous_rate':15,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 32,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'transition_phase_after':0.7
        }

hparams['naive_continuous_rate'] = 10
hparams['memorymaximum'] = 64
hparams['run_postfix'] = 2
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)
    
hparams['naive_continuous_rate'] = 10
hparams['memorymaximum'] = 64
hparams['run_postfix'] = 3
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)
    
    
hparams['naive_continuous_rate'] = 8
hparams['memorymaximum'] = 128
hparams['run_postfix'] = 2
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

naive_activecatinous74416
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous74416.job
Submitted batch job 3057555

naive_activecatinous79719
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous79719.job
Submitted batch job 3057556

naive_activecatinous50396
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous50396.job
Submitted batch job 3057557



In [6]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113 
# 1/10 = 170
# 1/8 = 212 with 0.00001 and 0.0001
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory':True,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 500,
        'run_postfix': 1,
        'memorymaximum': 64,
        'force_misclassified':True,
        'completion_limit': 8.17, #performance for first domain after base training
        'batch_size': 2,
        'training_batch_size': 8,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'len_perf_queue': 8,
         'transition_phase_after':0.7
        }

startbugdets = [85, 170, 212, 340]
memorymaximas = [64, 128]

sb=212
mm=128
i=0
#if i==0 and ((sb==85 and mm==128) or (sb==85 and mm==64) or (sb==170 and mm==64) or (sb==170 and mm==128) or (sb==212 and mm==128)):
#    pass
#else:
hparams['startbudget'] = sb
hparams['memorymaximum'] = mm
hparams['run_postfix'] = 1+i

if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

In [3]:
df_wrong_scanner = df_cache.loc[df_cache.filepath=='1.5T Philips']
df_wrong_scanner = df_wrong_scanner.rename(columns={'filepath':'scanner', 'scanner':'filepath'})
df_cache.loc[df_cache.filepath=='1.5T Philips', 'filepath'] = np.NaN
df_cache = df_cache.dropna().reset_index()
df_cache = df_wrong_scanner.append(df_cache)

In [2]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113
# 1/10 = 170
# 1/8 = 212
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'naive_activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
         'naive_continuous_rate':15,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 32,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'transition_phase_after':0.7
        }


continuous_rate = [20, 10, 8, 5]
memorymaximas = [64, 128]

for i in range(3):
    for cr in continuous_rate:
        for mm in memorymaximas:
            hparams['naive_continuous_rate'] = cr
            hparams['memorymaximum'] = mm
            hparams['run_postfix'] = i+1
            if schedule:
                slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
            else:
                model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

naive_activecatinous18009
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous18009.job
Submitted batch job 3045284

naive_activecatinous80834
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous80834.job
Submitted batch job 3045285

naive_activecatinous29483
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous29483.job
Submitted batch job 3045286

naive_activecatinous76348
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous76348.job
Submitted batch job 3045288

naive_activecatinous58108
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous58108.job
Submitted batch job 3045289

naive_activecatinous42727
sshpass -f ~/.ssh/pass ssh cn1.cir.medu

In [2]:
#experiments with startbudget of memory max 64 and 128
# 1/20 = 85
# 1/15 = 113
# 1/10 = 170
# 1/8 = 212
# 1/5 = 340

schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'naive_activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
         'naive_continuous_rate':15,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 10,
        'run_postfix': 1,
        'memorymaximum': 32,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
         'transition_phase_after':0.7
        }

hparams['naive_continuous_rate'] = 10
hparams['memorymaximum'] = 64
hparams['run_postfix'] = 2
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)
    
hparams['naive_continuous_rate'] = 10
hparams['memorymaximum'] = 64
hparams['run_postfix'] = 3
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)
    
    
hparams['naive_continuous_rate'] = 8
hparams['memorymaximum'] = 128
hparams['run_postfix'] = 2
if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams)

naive_activecatinous74416
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous74416.job
Submitted batch job 3057555

naive_activecatinous79719
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous79719.job
Submitted batch job 3057556

naive_activecatinous50396
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/naive_activecatinous50396.job
Submitted batch job 3057557



In [2]:
def test_model(model, ds, device):
    output = []
    target = []
    for data in ds:
        x, y, img, res = data
        x = x.float().to(device)
        y_out = model.forward(x)

        target.extend(y.detach().cpu().numpy())
        output.extend([o[0] for o in y_out.detach().cpu().numpy()])
        
        model.grammatrices = []
        
    return target, output

def test_hparams(hparams, dev='cuda'):
    device = torch.device(dev)
    test_phil15 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='test', res='1.5T Philips'), batch_size=4, num_workers=4)
    test_phil30 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='test', res='3.0T Philips'), batch_size=4, num_workers=4)
    test_30 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='test', res='3.0T'), batch_size=4, num_workers=4)
    
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams, train=False)
    
    target_15, output_15 = test_model(model, test_phil15, device)
    print('test phil 15', mean_absolute_error(target_15, output_15))
    target_30, output_30 = test_model(model, test_phil30, device)
    print('test phil 30', mean_absolute_error(target_30, output_30))
    target, output = test_model(model, test_30, device)
    print('test 30', mean_absolute_error(target, output))

In [3]:
hparams={'continuous': True,
         'use_memory':True,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 30,
        'run_postfix': 1,
        'memorymaximum': 64,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0
        }

In [4]:
hparams['startbudget'] = 340
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a4e882795d.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a4e882795d.pt
test phil 15 7.734721
test phil 30 8.738034
test 30 9.165479


In [5]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 340
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_10c966b4b1.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_10c966b4b1.pt
test phil 15 10.593702
test phil 30 9.637116
test 30 7.652421


In [4]:
hparams['startbudget'] = 212
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_f2015c6759.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_f2015c6759.pt
test phil 15 7.535403
test phil 30 9.165476
test 30 8.642139


In [4]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 212
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_976da7a5c0.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_976da7a5c0.pt
test phil 15 9.43274
test phil 30 9.942894
test 30 8.293361


In [4]:
hparams['startbudget'] = 212
hparams['learning_rate'] = 0.0001
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_0852e230e6.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_0852e230e6.pt
test phil 15 8.0726185
test phil 30 8.782521
test 30 8.608504


In [4]:
hparams['startbudget'] = 170
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_36e539216f.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_36e539216f.pt
test phil 15 8.222592
test phil 30 8.633889
test 30 9.691646


In [4]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 170
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_6d9047026b.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_6d9047026b.pt
test phil 15 10.407262
test phil 30 9.433815
test 30 8.557557


In [4]:
hparams['startbudget'] = 85
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a9eba22ef7.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a9eba22ef7.pt
No columns to parse from file
test phil 15 6.8602114
test phil 30 12.195896
test 30 12.864751


In [3]:
hparams={'continuous': True,
         'use_memory':True,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 30,
        'run_postfix': 1,
        'memorymaximum': 64,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0
        }

In [4]:
hparams['startbudget'] = 340
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a4e882795d.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a4e882795d.pt
test phil 15 7.734721
test phil 30 8.738034
test 30 9.165479


In [5]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 340
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_10c966b4b1.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_10c966b4b1.pt
test phil 15 10.593702
test phil 30 9.637116
test 30 7.652421


In [4]:
hparams['startbudget'] = 212
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_f2015c6759.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_f2015c6759.pt
test phil 15 7.535403
test phil 30 9.165476
test 30 8.642139


In [4]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 212
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_976da7a5c0.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_976da7a5c0.pt
test phil 15 9.43274
test phil 30 9.942894
test 30 8.293361


In [4]:
hparams['startbudget'] = 212
hparams['learning_rate'] = 0.0001
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_0852e230e6.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_0852e230e6.pt
test phil 15 8.0726185
test phil 30 8.782521
test 30 8.608504


In [4]:
hparams['startbudget'] = 170
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_36e539216f.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_36e539216f.pt
test phil 15 8.222592
test phil 30 8.633889
test 30 9.691646


In [4]:
hparams['memorymaximum'] = 128
hparams['startbudget'] = 170
test_hparams(hparams)

init
128
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_6d9047026b.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_6d9047026b.pt
test phil 15 10.407262
test phil 30 9.433815
test 30 8.557557


In [4]:
hparams['startbudget'] = 85
test_hparams(hparams)

init
64
/project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a9eba22ef7.pt
Read: /project/catinous/active_catinous/trained_models/cont_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_a9eba22ef7.pt
No columns to parse from file
test phil 15 6.8602114
test phil 30 12.195896
test 30 12.864751


In [3]:
hparams={'continuous': True,
         'use_memory': True,
         'naive_continuous': True,
        'datasetfile': '/project/catinous/brainds_split.csv',
        'base_model': 'batch_brainds_split_base_train_1_af4af4fb90.pt',
        'val_check_interval': 30,
        'run_postfix': 1,
        'memorymaximum': 64,
        'force_misclassified':True,
        'completion_limit': 6.5,
        'batch_size': 2,
        'training_batch_size': 4,
         'learning_rate': 0.00001,
         'random_cache': True,
         'gradient_clip_val': 0.5,
         'allowedlabelratio': 0,
        }

In [4]:
hparams['naive_continuous_rate'] = 5
test_hparams(hparams)

init
/project/catinous/active_catinous/trained_models/cont_naive_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_41354bd7b6.pt
Read: /project/catinous/active_catinous/trained_models/cont_naive_brainds_split_basemodel_brainds_fmiss_memory_tf08_1_41354bd7b6.pt
test phil 15 8.129477
test phil 30 10.051588
test 30 7.8155146


In [29]:
schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'jobname': 'baseline_activecont',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '240'}

hparams={'continuous':False,
         'use_memory':False,
         'datasetfile': '/project/catinous/brainds_split_val_test_balance_wo_ge.csv',
         'noncontinuous_train_splits': ['base_train', 'train'],
         'noncontinuous_steps': 4000,
         'root_dir': '',
         'batch_size': 2,
        'learning_rate': 0.0001}

if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams, train=False)

baseline_activecont02286
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/baseline_activecont02286.job
Submitted batch job 3075709



In [43]:
schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'jobname': 'baseline_activecont',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '240'}

hparams={'continuous':False,
         'use_memory':False,
         'datasetfile': '/project/catinous/brainds_split_val_test_balance.csv',
         'noncontinuous_train_splits': ['base_train', 'train'],
         'noncontinuous_steps': 2000,
         'root_dir': '',
         'batch_size': 2,
        'learning_rate': 0.0001,
        'run_postfix': 3,
        'scanner': '1.5T Philips'}

if schedule:
    slurm.srun(braincatsmodel.trained_model, [hparams], params=sparams, remote=True)
else:
    model, logs, df_cache, basemodel_lr = braincatsmodel.trained_model(hparams, train=False)

baseline_activecont32258
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/baseline_activecont32258.job
Submitted batch job 3076021



In [ ]:
device = torch.device('cuda')
hparams={'continuous':False,
         #'use_memory':False,
         'datasetfile': '/project/catinous/brainds_split.csv',
         'noncontinous_train_splits': ['base_train', 'train'],
         'noncontinous_steps': 5000,
         'root_dir': '',
         'batch_size': 2,
        'learning_rate': 0.0001}


model = FastGramDynamicMemoryBrainAge(hparams, device)
model.load_state_dict(torch.load('/project/catinous/active_catinous/trained_models/batch_brainds_split_base_train-train_1_861e8c8d6f.pt'))
model.eval()
val_phil15 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='val', res='1.5T Philips'), batch_size=4, num_workers=4)
val_phil30 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='val', res='3.0T Philips'), batch_size=4, num_workers=4)
val_30 = DataLoader(BrainAgeDataset('/project/catinous/brainds_split.csv', split='val', res='3.0T'), batch_size=4, num_workers=4)